In [ ]:
from pathlib import Path
import sys
from naver_cafe.setting import settings
from engine.sql.almaden import Sql
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time
import requests
import datetime
import pandas as pd
import pickle
from contextlib import suppress
import re
from html import unescape
db = Sql("datacast2")

In [2]:
def login(driver=None,username=None,password=None):
    driver.find_element_by_xpath('//*[@id="gnb_login_button"]').click()
    driver.find_element_by_xpath('//*[@id="id"]').send_keys(username)
    driver.find_element_by_xpath('//*[@id="pw"]').send_keys(password)
    driver.find_element_by_xpath('//*[@id="log.login"]').click()

In [3]:
def initiate_setting(task_keyword):
    setting_info = None
    try:
        setting_info = list(filter(lambda x : task_keyword in x.keys(),settings.info_list))[0][task_keyword]
    except Exception:
        print("task keyword 가 잘못되었습니다.")
    return setting_info

In [4]:
def select_task_id(task_keyword):
    row = db.select("crawl_task",'task_id','keyword=\"%s\" and channel=\"%s\"'%(task_keyword,"navercafe"))
    if len(row)>0:
        task_id = row[0]["task_id"]
    else:
        raise ValueError("task_id 에 해당하는 값이 없습니다.")
    return task_id

In [5]:
#setting
task_keyword = "삼국지전략판"  ###crawl_task 에서의 키워드
setting_info = initiate_setting(task_keyword)
task_id = select_task_id(task_keyword)
print(setting_info,task_id)
username = setting_info["username"]
password = setting_info["password"]
page_link = setting_info["link"]
login_required = setting_info["login_required"]
# 카페 정보
cafe = {'name': task_keyword                          # 카페 이름
       ,'page_link': page_link}

{'login_required': True, 'username': 'vasana12@naver.com', 'password': 'xmrwodud01!', 'link': 'https://cafe.naver.com/civilization4'} 156242


In [6]:
### 크롬 드라이버 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--dns-prefetch-disable")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

### 카페 url 로 이동
driver.get(cafe.get("page_link"))
if login_required:
    ### 로그인
    driver.implicitly_wait(10)
    login(driver=driver,username=username,password=password)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\wodud\.wdm\drivers\chromedriver\win32\98.0.4758.102]
C:\Users\wodud\AppData\Local\Temp\ipykernel_32368\3450633064.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="gnb_login_button"]').click()
C:\Users\wodud\AppData\Local\Temp\ipykernel_32368\3450633064.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="id"]').send_keys(username)
C:\Users\wodud\AppData\Local\Temp\ipykernel_32368\3450633064.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_b

In [7]:
num_per_page = 15          # 페이지당 게시글 갯수(default: 15개)
address_list=[]
page = 1

In [8]:
content_rows = db.select("crawl_contents","*",f"task_id={task_id} and crawl_status='F'")

i=0
contents_list = []   # 내용
reply_list = []      # 댓글
error_list = []      # 에러난 게시글

while True:
    ### 수집 링크로 이동
    url = content_rows[i]['url']
    contents_id = content_rows[i]['contents_id']
    driver.get(url)
    time.sleep(1)
    page_soup = None
    try:
        driver.switch_to.frame('cafe_main')
        r = driver.page_source
        page_soup = BeautifulSoup(r, "html.parser")
        content = page_soup.find('div', class_='ArticleContentBox')
        ### 게시글 수집
        temp_dict={}
        temp_dict['title'] = ""
        with suppress(AttributeError):   # 제목 없는 게시글
            temp_dict['title'] = content.find('h3',class_='title_text').text.strip()
        WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME,'article_viewer'))) #article_viewer 가 등장하기 전까지
        temp_dict['product_name'] = content.find("a",class_="link_board").text
        temp_dict['text'] = content.find("div", class_="CafeViewer").text.strip() ##중복되는 공지글 제외
        temp_dict['text'] = temp_dict['text'].replace("\n","")
        temp_dict['text'] = temp_dict['text'].replace("\u200b","")
        temp_dict['crawl_status'] = "T"
        print(f"location : {temp_dict['product_name']}/ text:{temp_dict['text']}")
        db.update_multi("crawl_contents",temp_dict,{"contents_id":contents_id})
        contents_list.append(temp_dict)

        ### 댓글 수집
        if content.find("div", class_="ReplyBox") is not None:   # 댓글 기능이 아예 없음
            comment_num = content.find("div", class_="ReplyBox").find("a",class_="button_comment").find("strong").text
            print(f"{i}번째:댓글 수집 확인 comment_num: {comment_num}")
            if comment_num!='0':   # 댓글이 있을떄
                comment = content.find("div", class_="CommentBox").find("ul",class_="comment_list").select("li")

                ### 댓글 구분
                com_n=0    # 댓글
                com_nn=0   # 대댓글
                comment_list = []
                for n in range(len(comment)):

                    if comment[n].get('class')==['CommentItem']:    # 댓글
                        com_n+=1; com_nn=0
                        com_thread = str(com_n)+"-"+str(com_nn)
                        com_nn=1
                    elif comment[n].get('class')==['CommentItem', 'CommentItem--reply']:    # 대댓글
                        com_thread = str(com_n)+"-"+str(com_nn)
                        com_nn+=1

                    ### 댓글 내용 수집
                    if comment[n].text.strip() != '삭제된 댓글입니다.':
                        com_nick = comment[n].find("a",class_="comment_nickname").text.strip()
                        com_date = comment[n].find("span",class_="comment_info_date").text.strip()
                        com_reply = comment[n].find("div",class_="comment_text_box").text.strip()
                        com_reply = com_reply.replace("\n","").replace("\u200b","")

                        # db.insert("crawl_contents",
                        #           author = com_nick,
                        #           task_id = task_id,
                        #           url=url,
                        #           text=com_reply,
                        #           is_reply=1)
                        com_reply_dict = {"author":com_nick, "text":com_reply}
                        comment_list.append(com_reply_dict)
                print(f"{i}//comment_list:{comment_list}")
                db.update_one("crawl_contents","review",str(comment_list),"contents_id", contents_id)
        i+=1

    except Exception as e:
        print(e)
        i+=1
        continue

#         ### 게시글을 볼 등급이 안됨
#         if page_soup.find('strong', class_='emph') is not None:
#             error_list.append({"error" :  page_soup.find('strong', class_='emph').text+"등급 필요"
#                                , "url" : url})
#             pass
#         ### 에러 따로 확인
#         else:
#             error_list.append({"error" : "에러 확인 필요"
#                                , "url" : url})
#             pass
# 크롬 종료

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A369A3+2582947]
	Ordinal0 [0x009CA6D1+2139857]
	Ordinal0 [0x008C3960+1063264]
	Ordinal0 [0x008B8092+1015954]
	Ordinal0 [0x008B88C8+1018056]
	Ordinal0 [0x008BA102+1024258]
	Ordinal0 [0x008B3DE9+998889]
	Ordinal0 [0x008C4DD0+1068496]
	Ordinal0 [0x00917B12+1407762]
	Ordinal0 [0x00908366+1344358]
	Ordinal0 [0x008E5176+1200502]
	Ordinal0 [0x008E6066+1204326]
	GetHandleVerifier [0x00BDBE02+1675858]
	GetHandleVerifier [0x00C9036C+2414524]
	GetHandleVerifier [0x00ACBB01+560977]
	GetHandleVerifier [0x00ACA8D3+556323]
	Ordinal0 [0x009D020E+2163214]
	Ordinal0 [0x009D5078+2183288]
	Ordinal0 [0x009D51C0+2183616]
	Ordinal0 [0x009DEE1C+2223644]
	Ba

KeyboardInterrupt: 